# Libraries

In [1]:
import ast
import math
import os
import gc

import cv2
from pathlib import Path

from PIL import Image, ImageSequence
import numpy as np
from numpy.fft import fft2, fftshift, ifft2, ifftshift
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, argrelextrema
# from skfda import FDataGrid
from numpy import unravel_index
import statistics
import heapq
import pandas as pd

# import tensorflow as tf
# from tensorflow.keras import layers, models

import itertools

from scipy.ndimage import gaussian_filter
from scipy.optimize import curve_fit
from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.patches as patches

# Functions

In [2]:
def interpolation(data):
    return(np.interp([i for i in range(1,len(data)*50)],xp=[i*50 for i in range(0,len(data))],fp=data))

In [3]:
def draw_circle(image,ccol,crow,circle_radius,result_folder,filename):
    fig, ax = plt.subplots()
    ax.imshow(image, cmap='gray')
    circle_center=(ccol,crow)
    circle = patches.Circle(circle_center, circle_radius, edgecolor='black', facecolor='none', linewidth=1)
    ax.add_patch(circle)

    cross_size = 10
    ax.plot([circle_center[0] - cross_size, circle_center[0] + cross_size], 
                [circle_center[1], circle_center[1]], color='white', linewidth=2) 
    ax.plot([circle_center[0], circle_center[0]], 
            [circle_center[1] - cross_size, circle_center[1] + cross_size], color='white', linewidth=2) 

    fig.savefig(os.path.join(result_folder, f"result_{filename}"))
    plt.cla()
    plt.close(fig)
    plt.close('all')
    del fig,ax


In [4]:
def process_image_low_movement(filename, output_path, preProcData, result_folder, i,df_list):
    i=i*50
    file_path = os.path.join(output_path, filename)

    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)


    movementNearFramesX=preProcData['diff_center_x'][i:i+100].sum()
    movementNearFramesY=preProcData['diff_center_y'][i:i+100].sum()

    if((movementNearFramesX+movementNearFramesY)>10):
        ccol=preProcData['center_y(ccol)'][i]
        crow=preProcData['center_x(crow)'][i]
    else:
        ccol=preProcData['smooth_center_y(ccol)'][i]
        crow=preProcData['smooth_center_x(crow)'][i]

    circle_radius = preProcData['circle_radius'][i]

    new_data = pd.DataFrame({
        'center_y(ccol)': [ccol],
        'center_x(crow)': [crow],
        'circle_radius': [circle_radius]
    })
    df_list.append(new_data)
    draw_circle(image, ccol, crow, circle_radius,result_folder,filename)
    # fig.imshow(fig, cmap='gray', vmin=0, vmax=255)
    
    
    # if(i%100==0):
    #     print(i)
    

In [5]:
def process_image_high_movement(filename, output_path, preProcData, result_folder, i,df_list):
    i=i*50
    file_path = os.path.join(output_path, filename)
    
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

    ccol=preProcData['center_y(ccol)'][i]
    crow=preProcData['center_x(crow)'][i]
    circle_radius = preProcData['circle_radius'][i]

    new_data = pd.DataFrame({
        'center_y(ccol)': [ccol],
        'center_x(crow)': [crow],
        'circle_radius': [circle_radius]
    })
    df_list.append(new_data)

    draw_circle(image, ccol, crow, circle_radius,result_folder,filename)
    # plt.imshow(fig, cmap='gray', vmin=0, vmax=255)

    
    # if(i%100==0):
    #     print(i)

<!-- 1 pixel ~ 0.032mikrometrai -->

# Main

In [6]:

def post_processing(test_number,r_in, r_out, ring_index, allowed_change, input_path, input_tiff, output_base_path,output_path,result_folder,excel_input_path,excel_path,rolling_window, interpolationP=True):
    # movement_threshold = 3500
    movement_threshold = 70
    # output_path = Path(output_base_path) / Path(input_tiff).stem
    # print(output_path)
    # output_path.mkdir(parents=True, exist_ok=True)
    test_number = input_tiff.split('.')[1]

    # result_folder = f"Results_Validation_Large_Interpolated_Mean/{test_number}/" + Path(input_tiff).stem + f"_Results_bw{r_in}{r_out}_{ring_index}"
    Path(result_folder).mkdir(parents=True, exist_ok=True)

    preProcData=pd.read_excel(excel_input_path)
    if (interpolationP==True):
        center_x_interpolated=interpolation(preProcData['center_x(crow)'])
        center_y_interpolated=interpolation(preProcData['center_y(ccol)'])
        circle_radius_interpolated=interpolation(preProcData['circle_radius'])
        window=50

        dfList=list(zip(center_x_interpolated,center_y_interpolated,circle_radius_interpolated))
        preProcData=pd.DataFrame(dfList,columns=['center_x(crow)','center_y(ccol)','circle_radius'])
    else:
        window = 3

    # calculating the sum of differences of the center (moving bead or stationary)
    preProcData['diff_center_y'] = preProcData['center_y(ccol)'].diff().abs().fillna(0)
    preProcData['diff_center_x'] = preProcData['center_x(crow)'].diff().abs().fillna(0)
    # preProcData['total_movement'] = preProcData['diff_center_y'] + preProcData['diff_center_x']
    totalMovement=0

    for index, row in preProcData.iterrows():
        totalMovement=totalMovement+row['diff_center_x']+row['diff_center_y']
    if rolling_window == 'mean':
        preProcData['smooth_center_y(ccol)'] = preProcData['center_y(ccol)'].rolling(window=window).mean()
        preProcData['smooth_center_x(crow)'] = preProcData['center_x(crow)'].rolling(window=window).mean()
        preProcData['smooth_radius'] = preProcData['circle_radius'].rolling(window=window).mean()
    else:
        preProcData['smooth_center_y(ccol)'] = preProcData['center_y(ccol)'].rolling(window=window).median()
        preProcData['smooth_center_x(crow)'] = preProcData['center_x(crow)'].rolling(window=window).median()
        preProcData['smooth_radius'] = preProcData['circle_radius'].rolling(window=window).median()

    preProcData.fillna(method='bfill', inplace=True) # for the beginning
    # .rolling(window=3)
    df = pd.DataFrame(columns=['center_y(ccol)', 'center_x(crow)', 'circle_radius'])

    totalMovement = preProcData['diff_center_x'].sum() + preProcData['diff_center_y'].sum()

    df_list=[]
    if totalMovement < movement_threshold:
        with ThreadPoolExecutor(max_workers=1) as executor:
            files = sorted(os.listdir(output_path))
            
            for i, filename in enumerate(files):
                executor.submit(process_image_low_movement, filename, output_path, preProcData, result_folder, i,df_list)
    else:
        with ThreadPoolExecutor(max_workers=1) as executor:
            files = sorted(os.listdir(output_path))
            for i, filename in enumerate(files):
                executor.submit(process_image_high_movement, filename, output_path, preProcData, result_folder, i,df_list)

    df = pd.concat(df_list, ignore_index=True)
    # print(df)
    df.to_excel(excel_path)


In [7]:
filters = [[3,7]]
for filter in filters:
    for test in range(1,11):
    # test='1'
        input_path='Tiffs_Validation'
        input_tiff=f'Video{test}.tif'
        # input_tiff=f'Video{test}.tif'
        r_in = filter[0]
        r_out = filter[1]
        r_in = 3
        r_out = 7
        ring_index = 1
        # brightness=image_background_brightness(get_first_png(input_tiff, input_path))
        params={
            'test_number':test,
            'r_in':r_in,
            'r_out':r_out,
            'ring_index':ring_index,
            'allowed_change':1,
            'input_path':input_path,
            'input_tiff':input_tiff,
            'rolling_window':'median',
            'output_base_path':f'Results_Validation/Video{test}',
            'output_path':f'../../Results_Validation/Video{test}/Video{test}',
            'result_folder' : f"../710centre37radiusMedian/Video{test}/" + Path(input_tiff).stem + f"_Results_bw{r_in}{r_out}_{ring_index}",
            'excel_input_path' : f'../ExcelsTesting/Video{test}_{r_in}{r_out}_{ring_index}_710center_37radius.xlsx',
            'excel_path':f'../ExcelsTesting/3637PostProcessed/Video{test}_{r_in}{r_out}_{ring_index}_710center_37radius_median.xlsx'
        }
        try:
            print(params)
            post_processing(**params)
        except Exception as e:
            print(e)
            continue
        # ring_search(**params)

{'test_number': 1, 'r_in': 3, 'r_out': 7, 'ring_index': 1, 'allowed_change': 1, 'input_path': 'Tiffs_Validation', 'input_tiff': 'Video1.tif', 'rolling_window': 'median', 'output_base_path': 'Results_Validation/Video1', 'output_path': '../../Results_Validation/Video1/Video1', 'result_folder': '../710centre37radiusMedian/Video1/Video1_Results_bw37_1', 'excel_input_path': '../ExcelsTesting/Video1_37_1_710center_37radius.xlsx', 'excel_path': '../ExcelsTesting/3637PostProcessed/Video1_37_1_710center_37radius_median.xlsx'}
[Errno 2] No such file or directory: '../ExcelsTesting/Video1_37_1_710center_37radius.xlsx'
{'test_number': 2, 'r_in': 3, 'r_out': 7, 'ring_index': 1, 'allowed_change': 1, 'input_path': 'Tiffs_Validation', 'input_tiff': 'Video2.tif', 'rolling_window': 'median', 'output_base_path': 'Results_Validation/Video2', 'output_path': '../../Results_Validation/Video2/Video2', 'result_folder': '../710centre37radiusMedian/Video2/Video2_Results_bw37_1', 'excel_input_path': '../ExcelsTes

C:\Users\Tautvydas\AppData\Local\Temp\ipykernel_5036\827899230.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  preProcData.fillna(method='bfill', inplace=True) # for the beginning


{'test_number': 3, 'r_in': 3, 'r_out': 7, 'ring_index': 1, 'allowed_change': 1, 'input_path': 'Tiffs_Validation', 'input_tiff': 'Video3.tif', 'rolling_window': 'median', 'output_base_path': 'Results_Validation/Video3', 'output_path': '../../Results_Validation/Video3/Video3', 'result_folder': '../710centre37radiusMedian/Video3/Video3_Results_bw37_1', 'excel_input_path': '../ExcelsTesting/Video3_37_1_710center_37radius.xlsx', 'excel_path': '../ExcelsTesting/3637PostProcessed/Video3_37_1_710center_37radius_median.xlsx'}


C:\Users\Tautvydas\AppData\Local\Temp\ipykernel_5036\827899230.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  preProcData.fillna(method='bfill', inplace=True) # for the beginning


{'test_number': 4, 'r_in': 3, 'r_out': 7, 'ring_index': 1, 'allowed_change': 1, 'input_path': 'Tiffs_Validation', 'input_tiff': 'Video4.tif', 'rolling_window': 'median', 'output_base_path': 'Results_Validation/Video4', 'output_path': '../../Results_Validation/Video4/Video4', 'result_folder': '../710centre37radiusMedian/Video4/Video4_Results_bw37_1', 'excel_input_path': '../ExcelsTesting/Video4_37_1_710center_37radius.xlsx', 'excel_path': '../ExcelsTesting/3637PostProcessed/Video4_37_1_710center_37radius_median.xlsx'}


...

C:\Users\Tautvydas\AppData\Local\Temp\ipykernel_5036\827899230.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  preProcData.fillna(method='bfill', inplace=True) # for the beginning


{'test_number': 9, 'r_in': 3, 'r_out': 7, 'ring_index': 1, 'allowed_change': 1, 'input_path': 'Tiffs_Validation', 'input_tiff': 'Video9.tif', 'rolling_window': 'median', 'output_base_path': 'Results_Validation/Video9', 'output_path': '../../Results_Validation/Video9/Video9', 'result_folder': '../710centre37radiusMedian/Video9/Video9_Results_bw37_1', 'excel_input_path': '../ExcelsTesting/Video9_37_1_710center_37radius.xlsx', 'excel_path': '../ExcelsTesting/3637PostProcessed/Video9_37_1_710center_37radius_median.xlsx'}


C:\Users\Tautvydas\AppData\Local\Temp\ipykernel_5036\827899230.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  preProcData.fillna(method='bfill', inplace=True) # for the beginning


{'test_number': 10, 'r_in': 3, 'r_out': 7, 'ring_index': 1, 'allowed_change': 1, 'input_path': 'Tiffs_Validation', 'input_tiff': 'Video10.tif', 'rolling_window': 'median', 'output_base_path': 'Results_Validation/Video10', 'output_path': '../../Results_Validation/Video10/Video10', 'result_folder': '../710centre37radiusMedian/Video10/Video10_Results_bw37_1', 'excel_input_path': '../ExcelsTesting/Video10_37_1_710center_37radius.xlsx', 'excel_path': '../ExcelsTesting/3637PostProcessed/Video10_37_1_710center_37radius_median.xlsx'}


C:\Users\Tautvydas\AppData\Local\Temp\ipykernel_5036\827899230.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  preProcData.fillna(method='bfill', inplace=True) # for the beginning


In [8]:
filters = [[3,7]]
for filter in filters:
    for test in range(1,11):
    # test='1'
        input_path='Tiffs_Validation'
        input_tiff=f'Video{test}.tif'
        # input_tiff=f'Video{test}.tif'
        r_in = filter[0]
        r_out = filter[1]
        r_in = 3
        r_out = 7
        ring_index = 1
        # brightness=image_background_brightness(get_first_png(input_tiff, input_path))
        params={
            'test_number':test,
            'r_in':r_in,
            'r_out':r_out,
            'ring_index':ring_index,
            'allowed_change':1,
            'input_path':input_path,
            'input_tiff':input_tiff,
            'rolling_window':'mean',
            'output_base_path':f'Results_Validation/Video{test}',
            'output_path':f'../../Results_Validation/Video{test}/Video{test}',
            'result_folder' : f"../710centre37radiusMean/Video{test}/" + Path(input_tiff).stem + f"_Results_bw{r_in}{r_out}_{ring_index}",
            'excel_input_path' : f'../ExcelsTesting/Video{test}_{r_in}{r_out}_{ring_index}_710center_37radius.xlsx',
            'excel_path':f'../ExcelsTesting/3637PostProcessed/Video{test}_{r_in}{r_out}_{ring_index}_710center_37radius_mean.xlsx'
        }
        try:
            print(params)
            post_processing(**params)
        except Exception as e:
            print(e)
            continue
        # ring_search(**params)

{'test_number': 1, 'r_in': 3, 'r_out': 7, 'ring_index': 1, 'allowed_change': 1, 'input_path': 'Tiffs_Validation', 'input_tiff': 'Video1.tif', 'rolling_window': 'mean', 'output_base_path': 'Results_Validation/Video1', 'output_path': '../../Results_Validation/Video1/Video1', 'result_folder': '../710centre37radiusMean/Video1/Video1_Results_bw37_1', 'excel_input_path': '../ExcelsTesting/Video1_37_1_710center_37radius.xlsx', 'excel_path': '../ExcelsTesting/3637PostProcessed/Video1_37_1_710center_37radius_mean.xlsx'}
[Errno 2] No such file or directory: '../ExcelsTesting/Video1_37_1_710center_37radius.xlsx'
{'test_number': 2, 'r_in': 3, 'r_out': 7, 'ring_index': 1, 'allowed_change': 1, 'input_path': 'Tiffs_Validation', 'input_tiff': 'Video2.tif', 'rolling_window': 'mean', 'output_base_path': 'Results_Validation/Video2', 'output_path': '../../Results_Validation/Video2/Video2', 'result_folder': '../710centre37radiusMean/Video2/Video2_Results_bw37_1', 'excel_input_path': '../ExcelsTesting/Video

C:\Users\Tautvydas\AppData\Local\Temp\ipykernel_5036\827899230.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  preProcData.fillna(method='bfill', inplace=True) # for the beginning


{'test_number': 3, 'r_in': 3, 'r_out': 7, 'ring_index': 1, 'allowed_change': 1, 'input_path': 'Tiffs_Validation', 'input_tiff': 'Video3.tif', 'rolling_window': 'mean', 'output_base_path': 'Results_Validation/Video3', 'output_path': '../../Results_Validation/Video3/Video3', 'result_folder': '../710centre37radiusMean/Video3/Video3_Results_bw37_1', 'excel_input_path': '../ExcelsTesting/Video3_37_1_710center_37radius.xlsx', 'excel_path': '../ExcelsTesting/3637PostProcessed/Video3_37_1_710center_37radius_mean.xlsx'}


C:\Users\Tautvydas\AppData\Local\Temp\ipykernel_5036\827899230.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  preProcData.fillna(method='bfill', inplace=True) # for the beginning


{'test_number': 4, 'r_in': 3, 'r_out': 7, 'ring_index': 1, 'allowed_change': 1, 'input_path': 'Tiffs_Validation', 'input_tiff': 'Video4.tif', 'rolling_window': 'mean', 'output_base_path': 'Results_Validation/Video4', 'output_path': '../../Results_Validation/Video4/Video4', 'result_folder': '../710centre37radiusMean/Video4/Video4_Results_bw37_1', 'excel_input_path': '../ExcelsTesting/Video4_37_1_710center_37radius.xlsx', 'excel_path': '../ExcelsTesting/3637PostProcessed/Video4_37_1_710center_37radius_mean.xlsx'}


...

C:\Users\Tautvydas\AppData\Local\Temp\ipykernel_5036\827899230.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  preProcData.fillna(method='bfill', inplace=True) # for the beginning


{'test_number': 9, 'r_in': 3, 'r_out': 7, 'ring_index': 1, 'allowed_change': 1, 'input_path': 'Tiffs_Validation', 'input_tiff': 'Video9.tif', 'rolling_window': 'mean', 'output_base_path': 'Results_Validation/Video9', 'output_path': '../../Results_Validation/Video9/Video9', 'result_folder': '../710centre37radiusMean/Video9/Video9_Results_bw37_1', 'excel_input_path': '../ExcelsTesting/Video9_37_1_710center_37radius.xlsx', 'excel_path': '../ExcelsTesting/3637PostProcessed/Video9_37_1_710center_37radius_mean.xlsx'}


C:\Users\Tautvydas\AppData\Local\Temp\ipykernel_5036\827899230.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  preProcData.fillna(method='bfill', inplace=True) # for the beginning


{'test_number': 10, 'r_in': 3, 'r_out': 7, 'ring_index': 1, 'allowed_change': 1, 'input_path': 'Tiffs_Validation', 'input_tiff': 'Video10.tif', 'rolling_window': 'mean', 'output_base_path': 'Results_Validation/Video10', 'output_path': '../../Results_Validation/Video10/Video10', 'result_folder': '../710centre37radiusMean/Video10/Video10_Results_bw37_1', 'excel_input_path': '../ExcelsTesting/Video10_37_1_710center_37radius.xlsx', 'excel_path': '../ExcelsTesting/3637PostProcessed/Video10_37_1_710center_37radius_mean.xlsx'}


C:\Users\Tautvydas\AppData\Local\Temp\ipykernel_5036\827899230.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  preProcData.fillna(method='bfill', inplace=True) # for the beginning
